![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

# Caminhos dos arquivos
filename_features = "features.txt"
filename_labels = "activity_labels.txt"

filename_subtrain = "subject_train.txt"
filename_xtrain = "X_train.txt"
filename_ytrain = "y_train.txt"

filename_subtest = "subject_test.txt"
filename_xtest = "X_test.txt"  # Corrigido aqui
filename_ytest = "y_test.txt"

# Carregamento dos dados
features = pd.read_csv(filename_features, header=None, names=['nome_var'], delim_whitespace=True)
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None)
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None)
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])


## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

In [14]:
%%time

def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean())/s.std()
    return s

X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)
X_train_pad.head()

CPU times: total: 188 ms
Wall time: 208 ms


,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.200628,-0.063678,-0.419600,-0.868755,-0.939377,-0.737479,-0.859758,-0.938955,-0.766385,-0.855978,...,-0.795305,0.025958,-0.276380,-0.360579,0.062935,-0.778374,-0.026079,-0.687172,0.407918,-0.007567
1,0.055944,0.031484,-0.253891,-0.875366,-0.923839,-0.849247,-0.868472,-0.921936,-0.848870,-0.871300,...,0.130605,-0.897296,-0.767938,0.133002,-0.021460,-1.218722,1.484369,-0.694091,0.409089,0.007875
2,0.073510,-0.043414,-0.076289,-0.868980,-0.907698,-0.893724,-0.863078,-0.898793,-0.896640,-0.863264,...,1.152257,-0.260860,-0.438286,-0.377815,0.391949,0.151197,1.704085,-0.702191,0.410260,0.026501
3,0.066691,-0.208407,-0.249695,-0.870566,-0.939959,-0.921743,-0.864445,-0.938060,-0.925216,-0.863264,...,1.112694,0.591005,0.463123,-0.135016,-0.033635,1.037781,-1.002951,-0.701636,0.414622,0.031712
4,0.030467,0.027585,-0.109840,-0.875128,-0.934815,-0.921281,-0.867325,-0.931726,-0.927965,-0.870201,...,-0.149567,-0.138505,-0.240296,0.340383,0.268468,1.125841,-1.276196,-0.700104,0.425434,0.045222


In [24]:
%%time

# PCA sem padronização
pca = PCA()
pca.fit(X_train)
var_exp = pca.explained_variance_ratio_ * 100
cum_var_exp = np.cumsum(var_exp)
cum_var_exp, var_exp

CPU times: total: 1.28 s
Wall time: 390 ms


(array([ 62.55443998,  67.4674627 ,  71.58893016,  73.46388628,
         75.15874627,  76.43081555,  77.60750069,  78.67647386,
         79.64585363,  80.50387181,  81.26617372,  81.93861938,
         82.51803897,  83.07591961,  83.57484534,  84.04978297,
         84.51698308,  84.94860094,  85.37431612,  85.78471116,
         86.17871356,  86.55402287,  86.90645036,  87.24580979,
         87.57794878,  87.89737757,  88.19915672,  88.49093929,
         88.78050925,  89.06243704,  89.33914119,  89.60253624,
         89.85784293,  90.09370881,  90.32436112,  90.54800929,
         90.77095742,  90.9812334 ,  91.18962632,  91.39440007,
         91.58725653,  91.77613615,  91.95731641,  92.13678911,
         92.30911678,  92.46931872,  92.62635821,  92.78298558,
         92.93595543,  93.08630671,  93.23142442,  93.37206458,
         93.50888964,  93.63574755,  93.76075366,  93.88049584,
         93.99861565,  94.11361054,  94.22669303,  94.33636258,
         94.44406665,  94.54896693,  94.

In [25]:
%%time
# PCA com padronização
pca_pad = PCA()
pca_pad.fit(X_train_pad)
var_exp_pad = pca_pad.explained_variance_ratio_ * 100
cum_var_exp_pad = np.cumsum(var_exp_pad)
cum_var_exp_pad, var_exp_pad

CPU times: total: 1.22 s
Wall time: 394 ms


(array([ 50.78117229,  57.36185256,  60.16828933,  62.67224208,
         64.56052709,  66.28453351,  67.65554498,  68.85462266,
         69.85048217,  70.81556876,  71.67562041,  72.47590136,
         73.23989773,  73.88522665,  74.517551  ,  75.11727309,
         75.70402339,  76.27943078,  76.84735183,  77.37464761,
         77.87501053,  78.36341894,  78.84162472,  79.31018765,
         79.75947691,  80.18050415,  80.59848284,  81.00405321,
         81.39257737,  81.77959542,  82.1455543 ,  82.50010768,
         82.84805028,  83.18523739,  83.51491439,  83.84312944,
         84.16365892,  84.45927386,  84.74599627,  85.03107082,
         85.29983714,  85.565457  ,  85.82886299,  86.08771359,
         86.33676846,  86.58372249,  86.82440241,  87.06051748,
         87.29079634,  87.51836358,  87.73852828,  87.95199527,
         88.15969972,  88.36219634,  88.56197578,  88.75972641,
         88.95400004,  89.1442372 ,  89.33230209,  89.51851806,
         89.6999846 ,  89.87736559,  90.

In [23]:
print("PCA sem padronização:")
print("Variância explicada por componente (%):", var_exp)
print("Variância explicada acumulada (%):", cum_var_exp)

PCA sem padronização:
Variância explicada por componente (%): [6.25544400e+01 4.91302272e+00 4.12146745e+00 1.87495612e+00
 1.69485999e+00 1.27206929e+00 1.17668514e+00 1.06897316e+00
 9.69379779e-01 8.58018171e-01 7.62301911e-01 6.72445661e-01
 5.79419596e-01 5.57880638e-01 4.98925730e-01 4.74937626e-01
 4.67200109e-01 4.31617860e-01 4.25715180e-01 4.10395039e-01
 3.94002401e-01 3.75309316e-01 3.52427488e-01 3.39359425e-01
 3.32138998e-01 3.19428786e-01 3.01779149e-01 2.91782575e-01
 2.89569956e-01 2.81927795e-01 2.76704148e-01 2.63395052e-01
 2.55306687e-01 2.35865883e-01 2.30652304e-01 2.23648168e-01
 2.22948133e-01 2.10275984e-01 2.08392917e-01 2.04773752e-01
 1.92856457e-01 1.88879623e-01 1.81180259e-01 1.79472696e-01
 1.72327672e-01 1.60201939e-01 1.57039496e-01 1.56627367e-01
 1.52969849e-01 1.50351278e-01 1.45117716e-01 1.40640152e-01
 1.36825069e-01 1.26857902e-01 1.25006115e-01 1.19742181e-01
 1.18119807e-01 1.14994893e-01 1.13082490e-01 1.09669551e-01
 1.07704072e-01 1.04900

In [22]:
print("\nPCA com padronização:")
print("Variância explicada por componente (%):", var_exp_pad)
print("Variância explicada acumulada (%):", cum_var_exp_pad)


PCA com padronização:
Variância explicada por componente (%): [5.07811723e+01 6.58068027e+00 2.80643677e+00 2.50395275e+00
 1.88828501e+00 1.72400642e+00 1.37101147e+00 1.19907768e+00
 9.95859511e-01 9.65086590e-01 8.60051649e-01 8.00280950e-01
 7.63996369e-01 6.45328924e-01 6.32324346e-01 5.99722090e-01
 5.86750302e-01 5.75407392e-01 5.67921047e-01 5.27295779e-01
 5.00362925e-01 4.88408406e-01 4.78205784e-01 4.68562930e-01
 4.49289260e-01 4.21027243e-01 4.17978682e-01 4.05570369e-01
 3.88524168e-01 3.87018044e-01 3.65958878e-01 3.54553381e-01
 3.47942606e-01 3.37187109e-01 3.29676999e-01 3.28215050e-01
 3.20529478e-01 2.95614939e-01 2.86722415e-01 2.85074548e-01
 2.68766324e-01 2.65619859e-01 2.63405989e-01 2.58850596e-01
 2.49054872e-01 2.46954034e-01 2.40679913e-01 2.36115075e-01
 2.30278857e-01 2.27567245e-01 2.20164699e-01 2.13466984e-01
 2.07704456e-01 2.02496620e-01 1.99779441e-01 1.97750625e-01
 1.94273627e-01 1.90237160e-01 1.88064891e-01 1.86215970e-01
 1.81466544e-01 1.7738

In [26]:
# Número de componentes para explicar pelo menos 90% da variância
n_components = np.where(cum_var_exp >= 90)[0][0] + 1
n_components_pad = np.where(cum_var_exp_pad >= 90)[0][0] + 1

In [27]:
print("\nNúmero de componentes necessárias para explicar 90% da variância:")
print("Sem padronização:", n_components)
print("Com padronização:", n_components_pad)


Número de componentes necessárias para explicar 90% da variância:
Sem padronização: 34
Com padronização: 63


## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

In [28]:
%%time
# Padronização dos dados de treino
X_train_pad = X_train.apply(padroniza, axis=0)

CPU times: total: 141 ms
Wall time: 209 ms


In [35]:
%%time

# PCA com 10 componentes - Dados não padronizados
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

X_train_pca, X_test_pca

CPU times: total: 375 ms
Wall time: 160 ms


(array([[-5.52028030e+00, -2.90277529e-01, -1.52992861e+00, ...,
          6.90966966e-01, -1.22229471e+00, -3.66474980e-01],
        [-5.53534954e+00, -8.25302526e-02, -1.92480424e+00, ...,
         -7.71428454e-01, -6.15518538e-01, -8.93861460e-01],
        [-5.47498801e+00,  2.87387048e-01, -2.14464153e+00, ...,
          9.44061766e-02, -6.31881287e-02, -2.13228672e-01],
        ...,
        [ 5.85750527e+00, -3.08184352e+00,  6.71207244e-01, ...,
         -1.63577930e-01,  2.92942207e-01,  1.24727516e+00],
        [ 5.42109482e+00, -3.42643018e+00,  6.71243196e-01, ...,
         -1.13274605e-01,  8.13250329e-01,  1.67764844e+00],
        [ 5.49797027e+00, -2.78992873e+00,  5.72221740e-03, ...,
         -8.42905493e-01,  9.97585187e-01,  1.81173523e-01]]),
 array([[-2.68674332, -1.21682132, -0.72207479, ...,  0.32741047,
          0.11767434, -1.59590609],
        [-4.33125548, -0.76632733, -1.12840395, ...,  0.80593566,
          0.59190338, -0.98597678],
        [-4.98536037,  0.

In [36]:
%%time

# PCA com 10 componentes - Dados padronizados
pca_pad = PCA(n_components=10)
X_train_pca_pad = pca_pad.fit_transform(X_train_pad)
X_test_pca_pad = pca_pad.transform(X_test.apply(padroniza, axis=0))

X_train_pca_pad, X_test_pca_pad

CPU times: total: 1.05 s
Wall time: 365 ms


(array([[-16.13744611,   2.15187766,   3.14457394, ...,  -4.90159753,
          -0.77716725,  -3.63653294],
        [-15.29515399,   1.38704944,  -0.68217755, ...,  -1.84478467,
           0.49113809,  -0.17593427],
        [-15.13598913,   2.47318271,  -1.75652528, ...,  -1.38859123,
           0.54135666,  -0.67049987],
        ...,
        [ 14.33246104, -12.25987812,   4.02565376, ...,  -0.88031599,
          -0.75954614,  -1.58436386],
        [ 12.87514324, -14.07029901,   2.91583614, ...,   0.31052432,
          -0.93218246,  -1.64781645],
        [ 13.01521842, -12.24342854,   1.33593009, ...,   2.61670412,
           0.777399  ,   0.02658117]]),
 array([[-10.94326227,  -1.86085345,   2.22773427, ...,  -2.4117442 ,
           3.15472227,   1.21758716],
        [-14.11149304,  -1.07770969,  -0.39663832, ...,   0.07467364,
          -0.54432715,   1.92696955],
        [-15.32164412,   2.36481476,  -2.25291245, ...,  -1.81027345,
           2.62689602,  -1.94090164],
        ...,


In [31]:
%%time

# Treinamento das árvores de decisão
dtc = DecisionTreeClassifier(ccp_alpha=0.001)
dtc_pad = DecisionTreeClassifier(ccp_alpha=0.001)

In [42]:
%%time

# Treino com dados não padronizados
dtc.fit(X_train_pca, y_train)

CPU times: total: 93.8 ms
Wall time: 93.9 ms


DecisionTreeClassifier(ccp_alpha=0.001)

In [43]:
%%time

# Treino com dados padronizados
dtc_pad.fit(X_train_pca_pad, y_train)

CPU times: total: 78.1 ms
Wall time: 86.4 ms


DecisionTreeClassifier(ccp_alpha=0.001)

In [44]:
%%time
# Avaliação da acurácia
train_accuracy = accuracy_score(y_train, dtc.predict(X_train_pca))
test_accuracy = accuracy_score(y_test, dtc.predict(X_test_pca))

train_accuracy, test_accuracy

CPU times: total: 0 ns
Wall time: 7 ms


(0.8920021762785637, 0.8218527315914489)

In [47]:
%%time

train_accuracy_pad = accuracy_score(y_train, dtc_pad.predict(X_train_pca_pad))
test_accuracy_pad = accuracy_score(y_test, dtc_pad.predict(X_test_pca_pad))

train_accuracy_pad, test_accuracy_pad

CPU times: total: 0 ns
Wall time: 7.01 ms


(0.8582698585418934, 0.7763827621309807)

In [48]:
print("Árvore com dados não padronizados:")
print(f"Acurácia de treinamento: {train_accuracy:.2f}")
print(f"Acurácia de teste: {test_accuracy:.2f}")

Árvore com dados não padronizados:
Acurácia de treinamento: 0.89
Acurácia de teste: 0.82


In [49]:
print("\nÁrvore com dados padronizados:")
print(f"Acurácia de treinamento: {train_accuracy_pad:.2f}")
print(f"Acurácia de teste: {test_accuracy_pad:.2f}")


Árvore com dados padronizados:
Acurácia de treinamento: 0.86
Acurácia de teste: 0.78


A diferença de acurácia entre árvores de decisão com dados padronizados e não padronizados geralmente se deve à forma como o PCA é afetado pela escala das variáveis:

Dados Não Padronizados: Variáveis com escalas maiores podem dominar o modelo, levando a uma performance inferior, pois o PCA pode enfatizar indevidamente a variância dessas variáveis em detrimento de outras mais informativas, mas de menor escala.

Dados Padronizados: Ao padronizar os dados, cada variável contribui igualmente, permitindo que o PCA identifique as direções de variância mais relevantes de forma mais efetiva. Isso geralmente melhora a acurácia do modelo, pois reduz o risco de sobreajuste e melhora a capacidade de generalização.